# notion-todo-sync sandbox
Look at how the code works, mainly as a reference for me to understand what the hell I've done and why (I hate web stuff 😿). Can't wait for Notion to refactor their API!!!

### Setup and init

In [ ]:
import requests as req

# for secrets
import os
from dotenv import load_dotenv
load_dotenv()

# for dotenv to work in IPython
%load_ext dotenv
%dotenv

## How it works
Execute the following every 20 seconds (this timing isn't dealt with in this script; we're CRON-jobbing for that):
1. Fetch the to-do database from Notion
2. Parse each page within to check if it has a `properties.Project.relation.id` key
   1. For pages that do, fetch from Notion the `properties.Origin.relation.id` from the `properties.Project.relation.id`
      1. Check against `properties.Origin.relation.id`
         1. If it matches, do nothing
         2. If it doesn't match, push to Notion the new `properties.Origin.relation.id`

## Limitations
- Does not work properly if the database has over 100 entries (Notion will only send first 100; requires extending to work)

## Code

### Create method which obtain's a page's `properties.Project`'s `properties.Origin` name

In [123]:
def parse_project_origin_name(project_id):
    # Fetch project page from Notion and extract only its 'Origin' property's page ID
    origin_id = req.get(
        f'https://api.notion.com/v1/pages/{project_id}',
        headers={
            'Authorization': f"Bearer {os.environ.get('NOTION_INTEGRATION_API_KEY')}",
            'Notion-Version': '2022-02-22'
        }
    ).json().get('properties').get('Origin').get('relation')[0].get('id')

    # Fetch origin page from Notion and extract only the page's plaintext name
    origin_name = req.get(
        f'https://api.notion.com/v1/pages/{origin_id}',
        headers={
            'Authorization': f"Bearer {os.environ.get('NOTION_INTEGRATION_API_KEY')}",
            'Notion-Version': '2022-02-22'
        }
    ).json().get('properties').get('Name').get('title')[0].get('plain_text')

    return origin_name

### Create method which updates a page's `properties.Origin` with a given new name

In [124]:
def set_new_origin_name(page_id, origin_name):
    req.patch(
        f'https://api.notion.com/v1/pages/{page_id}',
        headers={
            'Authorization': f"Bearer {os.environ.get('NOTION_INTEGRATION_API_KEY')}",
            'Notion-Version': '2022-02-22'
        },
        json={
            "properties": {
                "Origin": {
                    "select": {
                        "name": origin_name
                    }
                }
            }
        }
    )

### Fetch To-do DB from Notion

In [125]:
db = req.post(
    f'https://api.notion.com/v1/databases/{os.environ.get('NOTION_TARGET_DB_ID')}/query',
    headers={
        'Authorization': f"Bearer {os.environ.get('NOTION_INTEGRATION_API_KEY')}",
        'Notion-Version': '2022-02-22'
    },
    json={
        "filter": {
            "property": "Priority",
            "select": {
                "does_not_equal": "Completed"
            }
        }
    }
)

### Update pages in DB if they have missing or incorrect `properties.Origin` names 

In [126]:
db_pages = db.json()['results']

for page in db_pages:
    # If the page has a relation
    if page.get('properties').get('Project').get('relation'):
        project_id = page.get('properties').get('Project').get('relation')[0].get('id')
        current_origin_name = page.get('properties').get('Origin').get('select')

        # Fetch the correct origin name
        origin_name = parse_project_origin_name(project_id)

        # If the page's plaintext origin name doesn't match the project's plaintext origin name
        # OR the page has no plaintext origin name
        if (current_origin_name != origin_name or not current_origin_name):
            set_new_origin_name(page.get('id'), origin_name)

### We done!